In [ ]:
!ngrok authtoken # Your Authtoken Here

In [ ]:
import os
import sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

with HiddenPrints():
    # Prepare data
    DATA_ROOT = '/content/data'
    os.makedirs(DATA_ROOT, exist_ok=True)

    !pip -q install streamlit
    !pip -q install pyngrok
    # projet-specific installs
    !pip install deepface
    !pip install tf_keras
    !pip install opencv-python
    !pip install dlib
    !git clone https://github.com/antoinelame/GazeTracking.git

    from pyngrok import ngrok
    import streamlit
def launch_website():
    # Function to launch the Streamlit web app using ngrok for tunneling
    print("Click this link to try your web app:")
    if ngrok.get_tunnels():
        ngrok.kill()  # Kill existing ngrok tunnels
    tunnel = ngrok.connect()  # Open a ngrok tunnel to the streamlit app
    print(tunnel.public_url)  # Print only the public URL of the ngrok tunnel
    !streamlit run --server.port 80 app.py >/dev/null  # Launch the Streamlit app

In [ ]:
# Haar cascade .xml file for face detection
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

In [ ]:
%%writefile utils.py

import cv2
import numpy as np
from deepface import DeepFace
from google.colab.patches import cv2_imshow
from GazeTracking.gaze_tracking import GazeTracking
import imageio.v3 as iio


# Load Haar cascade classifier for face detection
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
gaze = GazeTracking()


# Emotion detection model

def detect_emotion (frame, emotion_count):
  # Convert frame to grayscale
  gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  # Convert grayscale to RGB
  rgb_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)
  # Detect face within frames
  faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.02, minNeighbors=1, minSize=(200,200))

  if len(faces) > 0:
    (x, y, w, h) = faces[0]
    # Find face ROI (Region of Interest)
    face_roi = rgb_frame[y:y + h, x:x + w]

    # Perform emotion detection on face ROI
    result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)

    # Determine dominant emotion in frame
    emotion = result[0]['dominant_emotion']

    # Track dominant emotions in frames
    if emotion in emotion_count.keys():
      emotion_count[emotion] += 1
    else:
      emotion_count[emotion] = 1

    # Outline and label face with predicted emotion
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
    cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)


# Gaze detection model

def detect_gaze (gaze, gaze_count):
  # Return frame with pupils highlighted
  frame = gaze.annotated_frame()
  text = ""

  # Track dominant gaze direction
  if gaze.is_right():
    text = "Looking right"
    gaze_count["right"] += 1
  elif gaze.is_left():
    text = "Looking left"
    gaze_count["left"] += 1
  elif gaze.is_center():
    text = "Looking center"
    gaze_count["center"] += 1

  # Label face with predicted gaze direction
  cv2.putText(frame, text, (60, 60), cv2.FONT_HERSHEY_DUPLEX, 2, (255, 0, 0), 2)
  cv2_imshow(frame)


# Logic fuction to determine focus level

def assess_focus(emotion_count, gaze_count, verbose=False):
  sorted_emotions = sorted(emotion_count, key = emotion_count.get, reverse=True)

  # Identify dominant emotion
  dominant_emotion = sorted_emotions[0]
  second_index = 1 if sorted(sorted_emotions[:2]) != sorted(['happy', 'neutral']) else 2
  next_to_dominant_emotion = sorted_emotions[second_index]

  # Get top score and second top score
  max_score = emotion_count[dominant_emotion]
  next_to_max_score = emotion_count[next_to_dominant_emotion]

  # Compare the two top scores to assess dominant emotion
  has_dominant_emotion = max_score > 1.1 * next_to_max_score

  # Assess emotion and gaze focus
  emotion_focused = has_dominant_emotion and dominant_emotion in ['happy', 'neutral']
  gaze_focused = gaze_count["center"] > gaze_count["left"] and gaze_count["center"] > gaze_count["right"]

  # Compare emotion and gaze focus to assess overall focus
  if emotion_focused and gaze_focused:
    verdict = "Nice focus! Let's move to the next level."
  elif emotion_focused or gaze_focused:
    verdict = "Let's continue with this level."
  else:
    verdict = "You seem to be distracted. Let's move down a level."

  if verbose:
    verdict += f"\n\nEmotion counts: {emotion_count}"
    verdict += f"\n\nGaze counts: {gaze_count}"

  return verdict



def categorize_video(video_filename):
  vid = cv2.VideoCapture(video_filename)

  emotion_count = {}
  gaze_count = {"center":0, "left":0, "right":0}

  frame_n = 0

  while True:
    # Capture video frame by frame
    ret, frame = vid.read()
    if frame is None:
      break

    # Read every 30 frames
    if frame_n%30==0:
      gaze.refresh(frame)

      # Run emotion detection
      detect_emotion(frame, emotion_count)

      # Run gaze detection
      detect_gaze(gaze, gaze_count)
    print(emotion_count)
    frame_n += 1

  # Release capture and close all windows
  vid.release()
  cv2.destroyAllWindows()

  # print("Here is your emotion analysis: " + str(emotion_count) + "\n")
  # print("Here is your gaze analysis: " + str(gaze_count) + "\n")

  # Run logic function to assess focus
  return assess_focus(emotion_count, gaze_count, verbose=True)

In [ ]:
%%writefile app.py

import streamlit as st
from utils import categorize_video
from PIL import Image

st.title('Adaptive Learning Through Emotion Detection and Gaze Tracking')

uploaded_file = st.file_uploader("Import Camera Video Here", type=["mov", "mp4"])

if uploaded_file is not None: # Runs only when user uploads video
    vid = uploaded_file.name
    with open(vid, mode='wb') as f:
        f.write(uploaded_file.read()) # Save video to disk

    st.video(data=uploaded_file, format=uploaded_file.type, start_time=0, subtitles=None, end_time=None, loop=False, autoplay=False, muted=False)
    response = categorize_video(vid)
    with st.chat_message("Your virtual teacher", avatar="🤖"):
        st.write(response)

In [ ]:
launch_website()